# Floating-point numbers: introduction

Equality is tested using `==` and `=` is reserved for variable substitution.

In [ ]:
a = 1 # variable a gets value 1

In [ ]:
a == 1 # check if a is 1

What does the following cell output?

In [ ]:
0.1 + 0.1 + 0.1 == 0.3

This notebook explains what happens here, but in short there are two things happening: computers store numbers in binary, but a finite decimal expnasion can be infinite in binary; and computers only store finitely many values (leading to truncation).

# Binary numbers

We are used to the decimal positional system
$$
\pi = 3.14159\dots
$$

Some rational numbers have infinite decimal expansion

$$
\frac 1 3 = 0.333\dots
$$
Computers use the binary positional system

$$
1 \equiv 1, \quad 2 \equiv 10, \quad 3 \equiv 11, \quad 4 \equiv 100, \quad \dots
$$

The integers are fine: any integer sequence in base-10 corresponds to a finite (though longer) sequence in binary and vice versa.

However, some rational numbers have finite decimal expansion but infinite binary expansion

$$
\frac 1 {10} \equiv 0.00011{\bf 0011}00110011\dots
$$

## Long division

Recall the [long division](https://en.wikipedia.org/wiki/Long_division) algorithm that you have learned in elementary school. 

In [ ]:
import numpy as np

def expand_ratio(x, y, max_steps = 10, base = 10):
    '''Expand x/y in a base (by default in the decimal base)'''
    q, r = divmod(x, y)
    output = np.base_repr(q, base)
    output += '.'
    for n in range(max_steps):
        if r == 0:
            break
        r = base*r
        q, r = divmod(r, y)
        output += np.base_repr(q, base)
    return output
    
print(expand_ratio(1, 10, base = 10))
print(expand_ratio(1, 10, base = 2))

# IEEE 754 floating-point numbers

This section is inspired by [Section 15](https://docs.python.org/3/tutorial/floatingpoint.html#floating-point-arithmetic-issues-and-limitations) of the excellent [Python Tutorial](https://docs.python.org/3/tutorial/index.html).

Infinite expansions of numbers need to be truncated in computations:

Numbers are stored using 53 bits (that is, binary digits) of precision (this part of the float is called the _mantissa_)
  * $2^0 = 1 \equiv 1$ can be stored using 1 bit, 
  * $2^1 = 2 \equiv 10$ and $3 \equiv 11$  can be stored using 2 bits, 
  * $2^2 = 4 \equiv 100, 5, 6$ and $7$  can be stored using 3 bits and so on. 

53 bits can store any integer between 0 and $2^{54} - 1$. Notice that even if we store the number 4, all bits are still reserved in the memory. So if large numbers are not needed, less bits should be reserved for each number.

In [ ]:
print(f'Largest integer stored with 53 bits: {2**54 - 1}')
print(f'Largest 8-bit unsigned (positive only) integer, `uint8`: {2**8 - 1}')
print(f'Largest 16-bit signed integer, `int16`: {2**15 - 1}')

The idea of floating-point numbers or floats is the same as [scientific notation](https://en.wikipedia.org/wiki/Scientific_notation): we separate the decimal part or significant figures and the (exponent of the) magnitude to get a logarithmic spacing of values covering both very small and very large numbers.

In [ ]:
from matplotlib import pyplot as plt

ax = plt.axes(xscale='log', yscale='linear', xlabel='Logarithmic scale', ylabel='Linear scale')
ax.grid()
ax.set_xlim((10**-5,10**3))
ax.set_ylim((0,10**3))
ax.yaxis.set_minor_locator(plt.MaxNLocator(25))


* Python uses [IEEE 754 floating-point](https://en.wikipedia.org/wiki/Double-precision_floating-point_format#IEEE_754_double-precision_binary_floating-point_format:_binary64) arithmetic, where every number is stored using 64 bits (this is also known as _double precision_, compared to _single precision_ which uses 32 bits, _half precision_ which uses 16 bits etc.). 
* One bit is reserved for the sign: $0 \sim$ positive, $1 \sim$ negative.
* 11 bits are used for the exponent
* 53 bits are used for the mantissa



On input 0.1 is converted to the closest number of the form $J 2^{-N}$ where 
$J$ and $N$ are integers that are normalized so that $2^{52} \le J < 2^{53}$. 

Let $x = 0.1$ and write
$x \approx J 2^{-N}$
for the best approximation with $J$ satisfying

$$
2^{52} \le J < 2^{53}.
$$

We replace $J$ by $x 2^N$ and take base 2 logarithm, denoted by $\log_2$,

$$
52 \le \log_2(x) + N < 53.
$$

This implies that $N$ must be chosen as the smallest integer satisfying 

$$
52 - \log_2(x) \le N.
$$

Equivalently, $N = 52 - n$ where $n$ is the largest integer satisfying

$$
n \le \log_2(x).
$$

In [ ]:
# Let's now solve for N and J
x = 0.1
n = np.floor(np.log2(x))
N = 52 - n
J = np.round(x * 2**N)
print(f'{N = }, {J = }')

In [ ]:
# Note that J is even so that we can also choose 
N = N - 1
J = J / 2
print(f'{N = }, {J = }')

The above normalization explains what _"Significand precision: 53 bits (52 explicitly stored)"_ means in the [IEEE 754 floating-point](https://en.wikipedia.org/wiki/Double-precision_floating-point_format#IEEE_754_double-precision_binary_floating-point_format:_binary64) article in Wikipedia.

To summarize, when we input 0.1, what the computer actually sees is 

$$
J 2^{-N} = 3602879701896397 \cdot 2^{-55},
$$

stored in binary. This number is not exactly 0.1. (It can not be since 0.1 has infinite expansion in binary.) 

To see the difference, we compare $J$ to
$$
x 2^N = 0.1 \cdot 2^N = \frac{2^N}{10}.
$$

In [ ]:
# J = 2^N/10 up to a small remainder
q, r = divmod(2**N, 10)
print(f'{q = }, {r = }, {J - q = }')

We see that $J$ was obtained by rounding up, and that $J 2^{-N}$ is slightly larger than $x = 0.1$. More precisely, 

$$
x 2^N = \frac{2^N}{10} = q + \frac{r}{10} = J-1 + \frac{8}{10} = J - \frac 1 5,
$$
and
$$
J 2^{-N} - x = \frac 1 5 2^{-N}.
$$

In [ ]:
# The rounding error is
err = 1/5 * 2**(-N)
err

# Limits of floating-points numbers

The limits of floating-point numbers are summarized in NumPy as follows

In [ ]:
print(np.finfo(float))
eps = np.finfo(float).eps
print(eps)

We rarely want to think in binary, and most of the above limits are given in decimal. For example, `eps` is the [machine epsilon](https://en.wikipedia.org/wiki/Machine_epsilon), that is, the difference between $1$ and the next smallest representable float larger than $1$. We write $x = 1 + \epsilon$ for this number. 
Let's compute $\epsilon$.

Analogously to the above, we let
$x = 1 + \epsilon = J 2^{-N}$.
Then $N = 52 - n$ where $n$ is the largest integer satisfying $n \le \log_2(x)$.
As $x = 1 + \epsilon$ is close to $1$, $\log_2(x)$ is close to $0$.
Hence $n = 0$ and $N = 52$. Now we can solve for $J$,

$$
J = 2^N x = 2^N + \epsilon 2^N.
$$

We want to find the smallest possible $\epsilon > 0$, while $J$ is an integer. Hence $J = 2^N + 1$ and 

$$
\epsilon = 2^{-N} = 2^{-52}.
$$

In [ ]:
np.finfo(float).eps == 2**(-52)

In [ ]:
# Let's return to our starting point and check that 
# 0.1 + 0.1 + 0.1 == 0.3
# up to floating-point precision
eps = np.finfo(float).eps
np.abs(0.1 + 0.1 + 0.1 - 0.3) < eps

# Rounding error

Let $x > 0$ be a real number, and let $J 2^{-N}$ be its best floating-point approximation. (In the case that there are two equally good approximations, we simply choose one of them.) We write again $n$ for the largest integer satisfying $n \le \log_2(x)$. Then $N = 52- n$ and

$$
|x 2^N - J| \le \frac 12.
$$

Therefore, using $2^n \le 2^{\log_2(x)} = x$,

$$
|x - J 2^{-N}| \le \frac 12 2^{-N} = \frac 12 2^{-52} 2^n \le \frac \epsilon 2 x,
$$

where $\epsilon$ is the machine epsilon. In other words, the _relative rounding error_,

$$
\frac{|x - J 2^{-N}|}{x},
$$

is at most half of the machine epsilon.

# Floating-point operations

Finite precision of floating-point numbers will affect the elementary operations $+,-,\cdot,/$. 

For simplicity, we consider a model of floating-point numbers that are stored using 8 decimal digits

$$
\mathbb F = \{ \pm J 10^{-N} : J, N \in \mathbb Z,\ 10^7 \le J < 10^8 \}.
$$

Note that we don't restrict here the size of the exponent $N$. In practice,
numbers with too small $-N$ will be rounded to zero, and too large $-N$ will cause an overflow error.

## Example: too small and large numbers

In [ ]:
# It is important to use 2.0, not 2, here.
# With 2, the integer data type is used and it has different limits. 
print(2.0**(-1074)) 
print(2.0**(-1075))
print(2.0**1023)
try:
    print(2.0**1024)
except OverflowError as e:
    print(e)

Addition is modelled by

$$
x \oplus y = \mathrm{fl}(x+y),
$$

where $\mathrm{fl}$ rounds to a nearest number in the set $\mathbb F$, and other floating-point operations are defined analogously. 

The relative error in floating-point operations, due to rounding, is at most half of the machine epsilon. When reasoning mathematically, this is sometimes called the fundamental axiom of floating point arithmetic.

In [ ]:
def fl(x, precision = 8):
    '''Simulate decimal floating-point rounding with given precision'''
    return float(np.format_float_positional(x, 
        precision = precision, fractional = False))
    
def oplus(x, y):
    return fl(x + y)

## Example: sum of three numbers

Let us compute $(a \oplus b) \oplus c$ and $a \oplus (b \oplus c)$ for $a,b,c \in \mathbb F$ defined by

$$
a = 0.23371258 \cdot 10^{-4}, 
\quad 
b = 0.33678429 \cdot 10^2,
\quad
c = -0.33677811 \cdot 10^2.
$$

In [ ]:
a = 0.23371258e-4
b = 0.33678429e2
c = -0.33677811e2
print(f'''
(1)  {oplus(oplus(a, b), c) = } 
(2)  {oplus(a, oplus(b, c)) = }
optimal = {fl(a + b + c)}
''')

As $b$ is much larger than $a$, only the first 3 digits of $a$ contribute to $a \oplus b$, the rest are lost in the rounding.

The second ordering gives the optimal result (obtained by computing the two sums in high precision and rounding only the end result). 



## Example: uneven sum

Many shortcomings are accounted by good implementations and those should be used whenever possible. Also note that math is usually even better than computitions.

In [ ]:
x = np.array([1.0 if i%10==0 else 10**(-5) for i in range(100)])

# Naive sum
S = 0
for s in x:
    S += s
print(f'Sum is {S}')
print(f'Numpy says {np.sum(x)}')
print(f'Math says {10*1.0 + 90*10**(-5)}')

## Example: continuous adding

In [ ]:
a = 1.9999999999999*2.0**52
print(f'{a = }')
for i in range(1000):
    if i%25 == 0:
        print(f'{i = }')
    b = a
    a += 1.0
    if a == b: # a + 1.0 should be bigger than b
        print(f'Underflow for {a = } when {i = }')
        break
# When did the underflow happen?
print(np.log2(a))

## Example: variance of small floats

Consider a uniformly distributed random variable $X \sim U(a,b)$. Given $n$ samples $x_i, i = 1,...,n$ the [variance](https://en.wikipedia.org/wiki/Variance) is given by
$$\text{Var}(X) = \frac{1}{n-1}\sum_{i=1}^n (x_i - \mu)^2 = \frac{1}{n-1}\Big(\sum_{i=1}^n x_i^2 - n \mu^2 \Big),$$
where $\mu$ is the mean.

If the mean $\mu \approx 0$ (say for small $a = -b$), then we can get underflow in the differences which leads to errors. However variance is shift-invariant:
$$\text{Var}(X) = \text{Var}(X - K),$$
meaning we can shift the values $x_i$ (and $\mu$) such that we get better precision.

In [ ]:
def Var(X):
    mu = np.mean(X)
    n = len(X)
    return (np.sum(X**2) - n*(mu**2))/(n-1)

n = 1000
a = 10**(-7)
X = np.random.uniform(-a, a, n)
print(f'K = 0: {Var(X)}')
K = X[0] # Common choice
print(f'K = x_0: {Var(X-K)}')
K = 1
print(f'K = 1: {Var(X - K)}')

# Implementation in numpy
print(f'numpy: {np.var(X, ddof=1)}')

# Catastrophic cancellation

In numerical analysis, catastrophic cancellation is the phenomenon that subtracting good approximations to two nearby numbers may yield a very bad approximation to the difference of the original numbers.

* Catastrophic cancellation may happen even if the difference is computed exactly
* It is not a property of any particular kind of arithmetic like floating-point arithmetic 
* Rather, it is inherent to subtraction, when the inputs are approximations themselves

However, rounding in floating-point operations may amplify the cancellation effect, as happened in case (1) in the above example. 

## Example: subtraction of measured quantities

This example is taken from [Catastrophic cancellation](https://en.wikipedia.org/wiki/Catastrophic_cancellation) Wikipedia article.

Consider two rods of lengths $L_1 = 254.5\,\text{cm}$ and $L_2 = 253.5\,\text{cm}$. If you measure them with a ruler that is good only to the centimeter, you might get approximations $\tilde L_1 = 255\,\text{cm}$ and $\tilde L_2 = 253\,\text{cm}$. 
The approximations are in relative error by less than 1% of the true lengths, 

$$
\frac{|L_1 - \tilde L_1|}{|L_1|} < 1\%.
$$

However, if you subtract the approximate lengths, you will get 

$$
\tilde L_1 - \tilde L_2 = 255\,\text{cm} - 253\,\text{cm} = 2\,\text{cm},
$$

even though the true difference between the lengths is 

$$
L_1 - L_2 = 254.5\,\text{cm} - 253.5\,\text{cm} = 1\,\text{cm}.
$$

The difference of the approximations is in relative error by 100% of the true difference.

## Example: rearranging computations

Sometimes it is possible to arrange the operations so that cancellation due to subtraction does not occur. Consider computing $e^x$ via truncation of its Taylor series

$$
e^x \approx 1 + x + \frac{x^2}{2!} + \dots \frac{x^n}{n!}.
$$

Let $x = -10$. We can either simply substitute to the above formula or first use

$$
e^{-10} = \frac{1}{e^{10}}
$$

and then substitute $x = 10$ in the formula.

In [ ]:
def exp_demo(x, n):
    '''Compute e^x by using its Taylor series up to the nth term
    
        1 + x + x^2/2! + ... + x^n/n!
    '''
    out = 1
    xn = 1
    cn = 1
    for i in range(1, n + 1):
        xn *= x
        cn /= i
        out += cn * xn
    return out

def relative_error(y):
    ytrue = np.exp(-10)
    return np.abs(y - ytrue) / ytrue

ns = range(1, 60)
err_left = [relative_error(exp_demo(-10, n)) for n in ns]
err_right = [relative_error(1 / exp_demo(10, n)) for n in ns]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(dpi = 150)
plt.semilogy(ns, err_left, 'r')
plt.semilogy(ns, err_right, 'b')

ax = plt.gca()
ax.set_xlabel('n');

# Common problems
By construction computing with floats can lead to truncation, cancellation, overflow and underflow. In particular the following things which are trivial or useful in mathematics become source of errors or problems in computations:
- Cancellation: subtraction of two nearby floats leaves only the most insignificant digits, rest is set to zero and after changing the exponent we have a very erroneous result.
    - In math it is common to state $a \neq b$ so take $(a - b) / 2$ etc. This does not necessarily work.
    - For example derivative: $\lim_{h \to 0} \frac{f(x + h) - f(x)}{h}$ will by defitition lead to cancellation and the limit will get *worse* as $h$ gets smaller.

- Converting back to integers: although floats can represent many integers perfectly, unexpected things can happen.
    - For example: `int(21.0/7.0) = 3` but `int(0.21/0.07) = 2`.

- Division with small number: in math one shouldn't divide by zero. In computations one shouldn't divide by a small number either. The latter is harder to check because *small* depends on the magnitude of the numerator. The end result may also be completely wrong but not overflow, whereas division by zero will usually lead to immediate error or `nan`.

- Testing for equality: similarly it may be impossible to get exact result so what is good enough? How much tolerance should there be? What is numerical error, what is implementation error?

- Hardware differences: computers have different types of floating-point units (FPUs). Some can do more than others, some can use more precision (bits). A tolerance which works in one computer might be too strict in another.

- Error propagation: small and indifferent error in one place can propagate and get amplified as it gets passed through the algorithm. Individual steps may be okay but the combination gives wrong results.
    - For example solving a differential equations vs. solving an equivalent integral equation.

# Cost in floating point operations (FLOPs)
It is common to compute the cost of a numerical method or algorithm with respect to [FLOPs](https://en.wikipedia.org/wiki/FLOPS). There are different ways of determining what constitutes 1 flop.

One way is to define it as single operation of a floating-point unit (FPU) which it can carry out in one cycle of the CPU clock. The basic operation is the *multiply - accumulate*
$$ a \leftarrow a + bc$$
This means that computing $a + b$, $a \times b$ and $a + b \times c$ are all equally expensive.

Modern processors can do more complicated things such as compute the dot product of short vectors (or their segments) in one cycle.

The mathematics way is to assign a cost of 1 flop to any basic artihmetic operation $+, -, \times , \div , \sqrt, \exp$. This is obviously wrong. It also makes measuring the cost super easy.

For example this example plot by Lincoln Atkinson from the blog post ["A simple benchmark of various math operations"](https://latkin.org/blog/2014/11/09/a-simple-benchmark-of-various-math-operations/) shows that the computational cost of different operations varies a lot. Note that it is normalized such that addition has cost of 1 unit.

![FLOPchart1.png](FLOPchart1.png)